In [7]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [8]:
url = "https://www.uiu.ac.bd/notice/"
mongo_string = "mongodb://localhost:27017/"

In [5]:
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
notice_card = soup.find_all("div", class_="notice")
len(notice_card)

12

In [12]:
client = MongoClient(mongo_string, 27017)
db = client["uiu_world"]
db_collection = db["notice"]

In [15]:
notice_card[0]

<div class="notice">
<div class="image">
<img alt="" src="https://www.uiu.ac.bd/wp-content/uploads/2024/04/UIU-Eid-Greetings-Card_Central.jpg"/>
</div>
<div class="details">
<!-- date -->
<div class="date-container">
<!-- icon -->
<span class="icon">
<img alt="" src="https://www.uiu.ac.bd/wp-content/themes/uiu-main/img/calendar-days-regular.svg"/>
</span>
<!-- date -->
<span class="date">
									April 3, 2024								</span>
</div>
<div class="title">
<a href="https://www.uiu.ac.bd/notice/holiday-notice-holy-shab-e-qadr-holy-eid-ul-fitr-bangla-nababarsha/">
									HOLIDAY NOTICE : HOLY SHAB-E-QAD’R, HOLY EID-UL-FIT’R &amp; BANGLA NABABARSHA								</a>
</div>
</div>
</div>

In [27]:
for notice in notice_card:
    date = notice.find("span", class_="date").text.strip().split(",")[0]
    title = notice.find("div", class_="title").text.strip().capitalize()
    notice_url = notice.select(".title > a")[0]["href"].strip()

    data = {"date": date, "title": title, "url": notice_url}

    db_collection.insert_one(data)

In [45]:
for notice in notice_card:
    res_details = requests.get(notice_card[3].select(".title > a")[0]["href"].strip())
    soup = BeautifulSoup(res_details.text, "html.parser")
    data_list = soup.find("div", class_="notice-details").text.strip().split("\n")
    data_list = [i for i in data_list if i != ""]
    data_list

['United International University will remain closed on March 26, 2024 (Tuesday) on the occasion of ‘INDEPENDENCE & NATIONAL DAY’.',
 'Dr. Md. Zulfiqur RahmanRegistrar']

In [51]:
for doc in db_collection.find():
    res_details = requests.get(doc["url"])
    soup = BeautifulSoup(res_details.text, "html.parser")
    data_list = soup.find("div", class_="notice-details").text.strip().split("\n")
    data_list = [i for i in data_list if i != ""]

    db_collection.update_one({"_id": doc["_id"]}, {"$set": {"details": data_list}})